## Simulação das Modalidades e Escalonamento FCFS

In [15]:
import threading
import time
import random
from sports import sports

# Função para simular a execução do esporte
def sport(title, duration, lock, wait_times):
    start_time = time.perf_counter()  # Captura o tempo inicial
    print(f'Modalidade {title} iniciada - tempo inicio: {start_time:.3f}')

    time.sleep(duration)  # Simula a execução do esporte pelo tempo de duração

    end_time = time.perf_counter()  # Captura o tempo final
    print(f'Modalidade {title} finalizada - tempo final: {end_time:.3f}')

    wait_time = end_time - start_time  # Calcula o tempo de espera
    with lock:
        wait_times.append(wait_time)

    print(f'Modalidade {title} concluída após {duration:.3f} horas. '
          f'Espera: {wait_time:.3f} segundos.')

# Escalonamento FCFS 
def escalonamento_fcfs(pending_sports, lock, wait_times):
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport, args=(title, duration, lock, wait_times))
        t.start()
        t.join()  # Executa o esporte até completar

# Simulação dos Jogos Olímpicos com FCFS
def start_olympics_fcfs():
    print('\nOs Jogos Olímpicos com FCFS foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados

    lock = threading.Lock()
    wait_times = []  # Lista para armazenar os tempos de espera

    # Adiciona esportes ao dia conforme há tempo disponível
    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    escalonamento_fcfs(pending_sports, lock, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (FCFS): {avg_wait_time:.3f} segundos\n')


start_olympics_fcfs()



Os Jogos Olímpicos com FCFS foram iniciados!

Modalidade Ciclismo Bmx Freestyle iniciada - tempo inicio: 1953821.921
Modalidade Ciclismo Bmx Freestyle finalizada - tempo final: 1953822.417
Modalidade Ciclismo Bmx Freestyle concluída após 0.496 horas. Espera: 0.497 segundos.
Modalidade Tiro Esportivo iniciada - tempo inicio: 1953822.425
Modalidade Tiro Esportivo finalizada - tempo final: 1953824.590
Modalidade Tiro Esportivo concluída após 2.164 horas. Espera: 2.165 segundos.
Modalidade Surfe iniciada - tempo inicio: 1953824.597
Modalidade Surfe finalizada - tempo final: 1953824.819
Modalidade Surfe concluída após 0.220 horas. Espera: 0.221 segundos.
Modalidade Ciclismo de Pista iniciada - tempo inicio: 1953824.827
Modalidade Ciclismo de Pista finalizada - tempo final: 1953829.250
Modalidade Ciclismo de Pista concluída após 4.423 horas. Espera: 4.424 segundos.
Modalidade Vôlei iniciada - tempo inicio: 1953829.258
Modalidade Vôlei finalizada - tempo final: 1953831.398
Modalidade Vôlei c

## Escalonamento Round Robin (RR)

In [16]:
# Escalonamento Round Robin
def escalonamento_rr(pending_sports, lock, wait_times, quantum=2):
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport, args=(title, min(duration, quantum), lock, wait_times))
        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

# Simulação dos Jogos Olímpicos com Round Robin
def start_olympics_rr():
    print('\nOs Jogos Olímpicos com Round Robin foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados

    lock = threading.Lock()
    wait_times = []  # Lista para armazenar os tempos de espera

    # Adiciona esportes ao dia conforme há tempo disponível
    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5) # Durações aleatórias

        pending_sports.append((title, duration))

    escalonamento_rr(pending_sports, lock, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (Round Robin): {avg_wait_time:.3f} segundos\n')

# Inicia a simulação com Round Robin
start_olympics_rr()


Os Jogos Olímpicos com Round Robin foram iniciados!

Modalidade Levantamento de Peso iniciada - tempo inicio: 1953962.570
Modalidade Levantamento de Peso finalizada - tempo final: 1953964.167
Modalidade Levantamento de Peso concluída após 1.596 horas. Espera: 1.597 segundos.
Modalidade Basquete 3x3 iniciada - tempo inicio: 1953964.174
Modalidade Basquete 3x3 finalizada - tempo final: 1953966.174
Modalidade Basquete 3x3 concluída após 2.000 horas. Espera: 2.001 segundos.
Modalidade Vôlei iniciada - tempo inicio: 1953966.181
Modalidade Vôlei finalizada - tempo final: 1953967.517
Modalidade Vôlei concluída após 1.336 horas. Espera: 1.336 segundos.
Modalidade Breaking iniciada - tempo inicio: 1953967.525
Modalidade Breaking finalizada - tempo final: 1953969.526
Modalidade Breaking concluída após 2.000 horas. Espera: 2.001 segundos.
Modalidade Taekwondo iniciada - tempo inicio: 1953969.530
Modalidade Taekwondo finalizada - tempo final: 1953971.531
Modalidade Taekwondo concluída após 2.000 

## Implementação dos Mecanismos de Sincronização

###  Semáforo Binário

In [17]:
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo Binário] Modalidade {title} aguardando acesso.')
    with binary_semaphore:
        print(f'[Semáforo Binário] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo Binário] Modalidade {title} liberou o acesso.')


### Monitor

In [18]:
monitor_lock = threading.Lock()

def sport_with_monitor(title, duration, daily_total_hours):
    print(f'[Monitor] Modalidade {title} aguardando acesso.')
    with monitor_lock:
        print(f'[Monitor] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Monitor] Modalidade {title} liberou o acesso.')


### Semáforo de Contagem

In [19]:
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso.')
    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso.')


## Escalonamento com Sincronismo

### FCFS + Semáforo

In [25]:
import threading
import time
import random
from sports import sports

lock = threading.Lock()

# Função original de execução do esporte, agora com semáforo binário
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo Binário] Modalidade {title} aguardando acesso.')
    with binary_semaphore:
        print(f'[Semáforo Binário] Modalidade {title} iniciou execução.')
        time.sleep(duration)  # Simula o tempo de execução do esporte
        with lock:
            daily_total_hours[0] += duration 
    print(f'[Semáforo Binário] Modalidade {title} liberou o acesso.')

# Escalonamento FCFS com semáforo binário
def escalonamento_fcfs(pending_sports, daily_total_hours, lock):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_binary_semaphore, args=(title, duration, daily_total_hours))
        executed_today.append((t, title))
        t.start()
        t.join()  # FCFS não permite interrupção, espera até terminar antes de iniciar o próximo

    return executed_today

# Simulação dos Jogos Olímpicos com FCFS e semáforo binário
def start_olympics_fcfs():
    print('\nOs Jogos Olímpicos com FCFS e semáforo binário foram iniciados!\n')

    day_duration = 12  # Duração de cada dia em horas
    day_count = 0  # Contador de dias
    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior

    while all_sports or pending_sports:
        day_count += 1
        print(f'--- Início do dia {day_count} ---\n')

        day_remaining = day_duration  
        daily_total_hours = [0]

        # Adiciona esportes ao dia conforme há tempo disponível
        while all_sports and day_remaining > 0:
            title = all_sports.pop(0)
            duration = random.uniform((1/60), 5)  # Gera um tempo aleatório entre 1 minuto e 5 horas

            if duration <= day_remaining:
                pending_sports.append((title, duration))
                day_remaining -= duration
            else:
                pending_sports.append((title, duration))
                break

        executed_today = escalonamento_fcfs(pending_sports, daily_total_hours, lock)
        
        executed_sports = ', '.join([title for _, title in executed_today])
        print(f'\n--- Fim do dia {day_count} ---\nEsportes executados: {executed_sports}\n')
        print(f'Total de horas de esportes executados no dia {day_count}: {daily_total_hours[0]:.3f} horas\n')
        
    print(f'Os Jogos Olímpicos foram finalizados após {day_count} dias!\n')

# Inicia a simulação com FCFS e semáforo binário
start_olympics_fcfs()



Os Jogos Olímpicos com FCFS e semáforo binário foram iniciados!

--- Início do dia 1 ---

[Semáforo Binário] Modalidade Atletismo aguardando acesso.
[Semáforo Binário] Modalidade Atletismo iniciou execução.
[Semáforo Binário] Modalidade Atletismo liberou o acesso.
[Semáforo Binário] Modalidade Badminton aguardando acesso.
[Semáforo Binário] Modalidade Badminton iniciou execução.
[Semáforo Binário] Modalidade Badminton liberou o acesso.
[Semáforo Binário] Modalidade Basquete aguardando acesso.
[Semáforo Binário] Modalidade Basquete iniciou execução.
[Semáforo Binário] Modalidade Basquete liberou o acesso.
[Semáforo Binário] Modalidade Basquete 3x3 aguardando acesso.
[Semáforo Binário] Modalidade Basquete 3x3 iniciou execução.
[Semáforo Binário] Modalidade Basquete 3x3 liberou o acesso.
[Semáforo Binário] Modalidade Boxe aguardando acesso.
[Semáforo Binário] Modalidade Boxe iniciou execução.
[Semáforo Binário] Modalidade Boxe liberou o acesso.
[Semáforo Binário] Modalidade Breaking agua

### RR + Semáforo de Contagem

In [27]:
import threading
import time
import random
from sports import sports

lock = threading.Lock()

# Função original de execução do esporte, agora com semáforo de contagem
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso.')
    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução.')
        time.sleep(duration)  # Simula o tempo de execução do esporte
        with lock:
            daily_total_hours[0] += duration 
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso.')

# Escalonamento Round Robin com semáforo de contagem
def escalonamento_rr(pending_sports, daily_total_hours, lock, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_counting_semaphore, args=(title, min(duration, quantum), daily_total_hours))
        executed_today.append((t, title))

        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

    return executed_today

# Simulação dos Jogos Olímpicos com Round Robin e semáforo de contagem
def start_olympics_rr():
    print('\nOs Jogos Olímpicos com Round Robin e semáforo de contagem foram iniciados!\n')

    day_duration = 12  # Duração de cada dia em horas
    day_count = 0  # Contador de dias
    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior

    while all_sports or pending_sports:
        day_count += 1
        print(f'--- Início do dia {day_count} ---\n')

        day_remaining = day_duration  
        daily_total_hours = [0]

        # Adiciona esportes ao dia conforme há tempo disponível
        while all_sports and day_remaining > 0:
            title = all_sports.pop(0)
            duration = random.uniform((1/60), 5)  # Gera um tempo aleatório entre 1 minuto e 5 horas

            if duration <= day_remaining:
                pending_sports.append((title, duration))
                day_remaining -= duration
            else:
                pending_sports.append((title, duration))
                break

        executed_today = escalonamento_rr(pending_sports, daily_total_hours, lock)
        
        executed_sports = ', '.join([title for _, title in executed_today])
        print(f'\n--- Fim do dia {day_count} ---\nEsportes executados: {executed_sports}\n')
        print(f'Total de horas de esportes executados no dia {day_count}: {daily_total_hours[0]:.3f} horas\n')

    print(f'Os Jogos Olímpicos foram finalizados após {day_count} dias!\n')

# Inicia a simulação com Round Robin e semáforo de contagem
start_olympics_rr()



Os Jogos Olímpicos com Round Robin e semáforo de contagem foram iniciados!

--- Início do dia 1 ---

[Semáforo de Contagem] Modalidade Atletismo aguardando acesso.
[Semáforo de Contagem] Modalidade Atletismo iniciou execução.
[Semáforo de Contagem] Modalidade Atletismo liberou o acesso.
[Semáforo de Contagem] Modalidade Badminton aguardando acesso.
[Semáforo de Contagem] Modalidade Badminton iniciou execução.
[Semáforo de Contagem] Modalidade Badminton liberou o acesso.
[Semáforo de Contagem] Modalidade Basquete aguardando acesso.
[Semáforo de Contagem] Modalidade Basquete iniciou execução.
[Semáforo de Contagem] Modalidade Basquete liberou o acesso.
[Semáforo de Contagem] Modalidade Basquete 3x3 aguardando acesso.
[Semáforo de Contagem] Modalidade Basquete 3x3 iniciou execução.
[Semáforo de Contagem] Modalidade Basquete 3x3 liberou o acesso.
[Semáforo de Contagem] Modalidade Boxe aguardando acesso.
[Semáforo de Contagem] Modalidade Boxe iniciou execução.
[Semáforo de Contagem] Modali

#### Comparação de resultado: 

FCFS será mais simples, com as modalidades executando uma por vez. O tempo total de execução será maior, pois nenhuma modalidade é interrompida.

Round Robin com semáforo de contagem permite execuções simultâneas, mas interrompe modalidades que ultrapassam o quantum, o que pode melhorar a eficiência em termos de utilização de recursos.

OBS.: O monitor não foi utilizado diretamente no código anterior porque, na prática, ele é bastante semelhante ao semáforo binário.